# Multiple Outputs (DDF)

 - Author : Sheng Dai (sheng.dai@aalto.fi)
 - Date : April 16, 2020

References:

[1] Kuosmanen, T., Zhou, X. and Dai, S. (2020). How much climate policy has cost for OECD countries?, World Development, 125, p. 104681.

[2] Kuosmanen, T. and Johnson, A. (2017). Modeling joint production of multiple outputs in StoNED: Directional distance function approach, European Journal of Operational Research. 262, pp. 792–801.

## Example

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the package pystoned
from pystoned import CNLSDDF

In [3]:
# import OECD countries data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/countries.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head(10)

,Country,Year,HRSN,CPNK,VALK,GHG
0,AUS,2010,19614.5,2952176.5,914377.0,537.2
1,AUS,2011,19949.9,3071779.7,951464.7,538.5
2,AUS,2012,20060.5,3192681.1,977274.9,541.3
3,AUS,2013,20181.6,3300697.4,1003749.5,531.3
4,AUS,2014,20359.3,3391029.7,1027981.2,525.8
5,AUS,2015,20684.5,3465641.2,1056251.5,533.3
6,AUT,2010,6827.0,1156202.4,263633.5,85.1
7,AUT,2011,6957.5,1171900.1,272240.3,82.7
8,AUT,2012,6929.5,1187259.8,273680.1,80.0
9,AUT,2013,6887.0,1202722.4,274071.2,80.2


In [4]:
# inputs
x1 = df['HRSN']
x1 = np.asmatrix(x1).T
x2 = df['CPNK']
x2 = np.asmatrix(x2).T
x = np.concatenate((x1, x2), axis=1)

# good output
y = df['VALK']

# bad output
b = df['GHG']

In [5]:
# define and solve the DEA-DDF model

fun = "prod"
gx  = [0.0, 0.0]
gy  = 1.0
gb  = -1.0

model = CNLSDDF.cnlsddfb(y, x, b, fun, gx, gb, gy)

# using local solver (MOSEK API)
from pyomo.opt import SolverFactory
opt = SolverFactory("mosek")
results = opt.solve(model, tee=True)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 28392           
  Cones                  : 0               
  Scalar variables       : 1008            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.01    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 80
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0              

In [6]:
# display the estimates (alpha, beta, gamma, delta, and residual)
model.a.display()
model.b.display()
model.g.display()
model.d.display()
model.e.display()

a : alpha
    Size=168, Index=i
    Key : Lower : Value               : Upper : Fixed : Stale : Domain
      0 :  None :  -378490.1412240376 :  None : False : False :  Reals
      1 :  None :   316972.2906013991 :  None : False : False :  Reals
      2 :  None :  1106821.2890560352 :  None : False : False :  Reals
      3 :  None :  2094370.5111410213 :  None : False : False :  Reals
      4 :  None :  3505268.2036937196 :  None : False : False :  Reals
      5 :  None :   5338304.480334093 :  None : False : False :  Reals
      6 :  None :  -5087464.558872428 :  None : False : False :  Reals
      7 :  None :  -4467788.696666822 :  None : False : False :  Reals
      8 :  None :  -4574336.566740076 :  None : False : False :  Reals
      9 :  None :  -4976270.334573097 :  None : False : False :  Reals
     10 :  None :  -4673596.774087743 :  None : False : False :  Reals
     11 :  None :  -5045070.962588059 :  None : False : False :  Reals
     12 :  None :  -5777764.746472648 :  None

In [7]:
# retrive the alpha
val = list(model.a[:].value)
alpha = np.asarray(val)
alpha

array([  -378490.14122404,    316972.2906014 ,   1106821.28905604,
         2094370.51114102,   3505268.20369372,   5338304.48033409,
        -5087464.55887243,  -4467788.69666682,  -4574336.56674008,
        -4976270.3345731 ,  -4673596.77408774,  -5045070.96258806,
        -5777764.74647265,  -5076382.61698882,  -5052816.98848527,
        -5246882.35896191,  -5023098.48028029,  -4809037.32009486,
        10020037.63383231,  13673631.45725167,  11725277.4230298 ,
        13101498.83512696,  14888639.31505085,  11650622.55447575,
        -3893252.76773241,  -3634316.15660251,  -3621782.98522471,
        -3648474.38125963,  -3119867.52218558,  -3114414.13354107,
        -7835347.96286198,  -7414224.27512989,  -7376322.29452858,
        -6961976.40907757,  -6606946.96412426,  -6030957.68600765,
       -10909404.87603062, -10382422.60310882, -10463113.9511105 ,
       -10571778.44015072, -10634634.7671901 , -10606277.04220166,
        -8024825.94941701,  -7828219.54869675,  -7784680.12413

In [8]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)
eps

array([ 2.55620213e+06,  1.61468491e+06,  6.33839673e+05, -7.57621361e+05,
       -2.09296064e+06, -3.36166004e+06, -5.86266461e+04,  4.27387311e+01,
       -4.18989031e+04, -9.60464841e+04, -1.06127654e+05, -2.35640905e+05,
       -1.93211768e+06, -1.94380151e+06, -1.81333589e+06, -1.85555836e+06,
       -2.02560956e+06, -2.29006485e+06, -2.52972123e+06, -3.34826540e+06,
       -2.73588825e+06, -2.99240535e+06, -3.93450524e+06, -3.18636791e+06,
        1.48827154e+06,  1.54645172e+06,  1.57760872e+06,  1.60397776e+06,
        1.67121627e+06,  1.69539070e+06, -5.03069268e+06, -5.06470283e+06,
       -5.31058168e+06, -5.40338670e+06, -5.61211205e+06, -5.65034523e+06,
       -8.79671869e+06, -8.64849421e+06, -8.63143145e+06, -8.61352217e+06,
       -8.59559231e+06, -8.51515827e+06, -3.22130133e+06, -3.21626332e+06,
       -3.10908714e+06, -3.16276829e+06, -3.18780355e+06, -3.19185265e+06,
        1.32937034e+07,  1.28378185e+07,  1.25182749e+07,  1.14342432e+07,
        1.05756773e+07,  

In [9]:
# retrive the beta
ind = list(model.b)
val = list(model.b[:, :].value)
beta = np.asarray([i + tuple([j]) for i, j in zip(ind, val)])

beta = pd.DataFrame(beta, columns=['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['HRSN', 'CPNK']
beta

,HRSN,CPNK
Name,,
0.0,1975.635326,7.680823
1.0,2564.327211,6.860597
2.0,3151.791222,5.161419
3.0,3830.903841,2.406449
4.0,4354.948410,0.937183
...,...,...
163.0,98.779211,0.384206
164.0,49.010215,1.031434
165.0,56.923620,1.737096


In [11]:
# retrive the delta
val = list(model.d[:].value)
delta = np.asarray(val)
delta

array([ 63.15964917,  73.74920217,  81.2448595 ,  86.97947334,
        93.98939816, 103.54315233,   6.13094907,   8.70326719,
        11.07659767,  13.26993142,  16.3310014 ,  20.58533034,
        58.52113732,  72.02690376,  61.36290819,  58.81896442,
        65.74861639,  73.78246149,  70.03229751,  82.81673085,
        74.46559331,  79.46921697,  91.42375339,  80.06899245,
         1.77658826,   1.86840173,   1.26929109,   0.63716224,
         0.58897063,   1.32152627,  43.45472764,  48.70373353,
        54.40468488,  60.26202913,  66.91794619,  72.10150957,
         7.27291844,  11.5243841 ,  18.6870294 ,  12.392552  ,
        13.81710925,   5.88666592,  23.91672493,  30.23448574,
        16.79347584,  15.56891371,  12.74199789,  12.124153  ,
        28.27488165,  34.02873054,  38.15530016,  43.37902521,
        49.06645976,  55.0515594 ,   6.16160164,   8.96490479,
        10.59196427,  13.37564063,  16.374975  ,  19.87358888,
        18.62786879,  11.67146235,   8.48581949,   5.55

In [12]:
# retrive the gamma
val = list(model.g[:].value)
gamma = np.asarray(val)
gamma

array([ 64.00585579,  74.59479517,  82.08768222,  87.82563132,
        94.83574788, 104.38930348,   6.97593354,   9.54735237,
        11.92006765,  14.11406276,  17.17353248,  21.42908416,
        59.37045574,  72.8732993 ,  62.20915738,  59.6660064 ,
        66.59420673,  74.62836316,  70.8792374 ,  83.66386638,
        75.31275993,  80.31652512,  92.27057295,  80.91556358,
         2.62445295,   2.71514756,   2.11936702,   1.47328673,
         1.43155316,   2.16709732,  44.30188321,  49.54961509,
        55.24971468,  61.10775766,  67.76296246,  72.94590223,
         8.11927154,  12.37114512,  19.53366355,  13.23965206,
        14.66398163,   6.72844832,  24.76473845,  31.08139041,
        17.63940174,  16.41616878,  13.58788063,  12.96774939,
        29.11454531,  34.86665815,  38.99348341,  44.21807335,
        49.90408689,  55.88963576,   7.00280133,   9.8037776 ,
        11.43134179,  14.21671088,  17.2121668 ,  20.71032533,
        19.47201176,  12.51625924,   9.33120122,   6.39